In [ ]:
!pip install apache_beam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.9 MB 2.0 MB/s 
     |████████████████████████████████| 47 kB 4.7 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 270 kB 57.6 MB/s 
     |████████████████████████████████| 151 kB 48.0 MB/s 
     |████████████████████████████████| 508 kB 58.8 MB/s 
     |████████████████████████████████| 2.4 MB 45.9 MB/s 
     |████████████████████████████████| 1.0 MB 54.6 MB/s 
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=f7872786ebd168a8113c41a09e0919fa6827bf50f0784404074a8604e2c97941
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=ce851522177b1cebc4b65f382758c117e0fc6f96e5e5e456819f281b272dc81d
  Stored in directory: /root/.ca

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.options.pipeline_options import PipelineOptions

Para as configurações do pipeline criearemos um objeto chamado pipeline_options e instanciar um objeto da classe que acabamos de importar

In [ ]:
pipeline_options = PipelineOptions(argv = None)

In [ ]:
pipeline = beam.Pipeline(options = pipeline_options)

 Nesse trecho de código, nessas quatro linhas que digitamos, nós importamos o Apache Beam para o nosso projeto, importamos as opções de pipeline para o nosso projeto, nosso código aqui, e nós criamos dois objetos, que são as opções de pipeline, instanciando a classe que nós acabamos de importar e também criamos um objeto chamado pipeline que recebeu como parâmetro essas opções de pipeline.

 O Apache Beam, esse que acabamos de importar, é um modelo de programação unificado de código aberto para definir e executar pipelines de processamento de dados, por exemplo, o famoso ETL, que é a extração, a transformação e o carregamento de dados.

[04:17] A SDK do Apache Beam permite que nós façamos processamento tanto em lote, conhecido como batch, ou seja, grande quantidade de arquivos brutos que nós vamos processar linha a linha, em partes, como também os famosos streams, ou então processamentos em fluxo. Mas durante o nosso processo, vamos utilizar apenas o processamento em batch.

In [ ]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) #pular a primeira linha com o skip_header

)

a variável dengue é a pcollection que possui o processo de ler o arquivo, a variável dengue contem os resultados de processo das pipelines que ela recebe e nesse caso é o casos_dengue.txt

Esse meu método da SDK, o ReadFromText, vai receber o local do arquivo que ele vai ler o arquivo, nós colocamos aqui o nome dele, casos_dengue, que está no mesmo nível, porém eu podia colocar aqui um arquivo que está vindo de um bucket, de um storage, do GCP, da AWS, não importa, então eu colocaria aqui esse endereço.

# Criar algorítimos que facilitam o acesso a dados brutos

Seria interessante que transformássemos todos esses dados em uma lista ou tupla para que seja mais fácil de tratar esses dados e visivelmente melhor. Vamos então, criar um método que receba essas informações e converta ela em uma lista e usaremos o separador para delimitar esses dados.

Vamos criar um método e aproveitar que nosso arquivo, até então, é lido linha a linha e converteremos elas em lista

In [ ]:
def TextoParaLista(elemento,delimitador = '|'):
  """
  Recebe um texto e um limitador
  Retorna uma lista de elementos pelo delimitador
  """
  return elemento.split(delimitador)

In [ ]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) #pular a primeira linha com o skip_header
    | "De texto para lista"  >> beam.Map(TextoParaLista)
    | "Mostrar resultados" >> beam.Map(print)
)

pipeline.run() 

Ao rodar o código acima teremos o codigo rodando por um bom tempo mostrando linha a linha agora em formato de lista.

Próximo passo será armazenar os dados em dicionários.

# Transformando nossa pcollection em dicionário

In [ ]:
colunas_dengue = [ #todas as colunas que vem do dataset
'id',
'data_iniSE',
'casos',
'ibge_code',
'cidade',
'uf',
'cep',
'latitude',
'longitude']

In [ ]:
#PARA RELEMBRAR USO DO ZIP EM PYTHON
#O ZIP TRANSFORMAR DUAS LISTAS EM UM DICIONÁRIO

lista1 = ['a','b','c','d']
lista2 = [1,2,3,4]
dicionario = dict(zip(lista1,lista2))
print(dicionario)
print(dicionario.get('a'))
print(dicionario.get('c'))
print(dicionario.get('e')) #Resultado none pois não existe chave 'e' 

In [ ]:
def ListaParaDicionario(elemento, colunas):
  """
  Recebe duas listas e retorna um dicionário
  """
  return dict(zip(colunas, elemento))

In [ ]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) #pular a primeira linha com o skip_header
    | "De texto para lista"  >> beam.Map(TextoParaLista)
    | "Lista para dicionário" >> beam.Map(ListaParaDicionario, colunas_dengue)
    | "Mostrar resultados" >> beam.Map(print) #Caso queira conferir descomente a linha
)

pipeline.run()



Por exemplo, vamos aplicar transformação na data, então para pegar a data eu já sei quem é, eu pego segundo o dicionário, com a chave, que é a data do início da semana epidemiológica.
Agora que temos esse nosso dicionário, essa minha pcollection dengue tem o resultado, que é um dicionário com chave e valor para cada coluna, eu posso agora começar a aplicar as nossas transformações que planejamos inicialmente.

# Tratamento de dados

Aqui no dataset de chuvas nós vamos ter data completa, dia a dia, para o mesmo estado eu tenho várias leituras do mesmo dia. Já no dataset de casos de dengue eu não vou ter isso, eu vou ter uma data de início da semana epidemiológica completa aqui, por exemplo, 8 de novembro de 2015, nós temos a quantidade de casos naquela semana, naquela cidade, naquele estado.

Essas informações aqui nós não temos no dataset de chuvas, então ficou definido que nós iríamos utilizar o que eles têm em comum. Nós temos como utilizar a quantidade aqui definida pelo mês e pelo ano, nós podemos acumular a quantidade de chuva mês, ano, estado.

Então nós vamos ter que fazer essa transformação aqui no dataset de dengue, nós vamos ter que pegar essa data, transformar e acumular pelo mês e pelo ano, para que possamos também fazer isso em chuvas e assim podemos correlacionar e juntar as duas bases.

Vamos então:
 

*   Unificar os dados por ano e mês
*   Criar um novo campo composto por esses valores (Isso é chamado hash)
Removeremoses o dia e criaremos um novo campo (hash) somente com mês e ano



In [ ]:
def TrataDatas(elemento):
  """
  Recebe um dicionário e cria um novo campo com ano-mês
  """
  elemento['ano-mes'] = '-'.join(elemento['data_iniSE'].split('-')[:2])
  #Supondo que temos a data 2022-05-18 o split retornará uma lista com ['2022','05','18']
  #Então fariaremos a lista usando um [:2] no final pegando as informações até o segundo elemento
  #Aplicamos também o join com o parâmetro '-' que nos retornará uma STRING no formato 2022-05
  return elemento

In [ ]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) #pular a primeira linha com o skip_header
    | "De texto para lista"  >> beam.Map(TextoParaLista)
    | "Lista para dicionário" >> beam.Map(ListaParaDicionario, colunas_dengue)
    | "Criar campo ano-mês" >> beam.Map(TrataDatas)
    | "Mostrar resultados" >> beam.Map(print) #Caso queira conferir descomente a linha

)

pipeline.run()

Esse hash que acabamos de criar, com ano e mês, vai ser muito importante nos próximos passos, já que vamos utilizá-lo como chave para agrupar os elementos e assim conseguir agregar os dados nessa abstração. Que abstração é essa? É conseguir acumular a quantidade de casos de dengue por ano e por mês. E também nós vamos fazer a mesma coisa com o dataset de chuvas.

# Organização por estado

amos retornar ao nosso dataset de chuvas, demorou um pouco para carregar porque, lembra, ele tem mais de 25 milhões de linhas se não me engano, ele tem uma coluna que são os estados, a coluna uf. E no dataset de casos de dengue nós temos também essa coluna de estado.
Uma forma de nós agruparmos essas informações, de contabilizar, seria pelo estado. Nós já fizemos uma nova chave de ano e mês, e agora podemos fazer um agrupamento por ano, mês e estado. Assim vamos contabilizar a quantidade de casos por mês, ano e por estado, e assim vamos poder juntar os dois datasets seguindo a mesma lógica, eu vou ter aqui a quantidade de chuvas em milímetros por mês, ano e estado, e aqui eu vou ter a quantidade de casos de dengue por ano, mês e também pelo estado.

In [ ]:
def ChaveUf(elemento):
  """
  Receber um dicionário
  Retornar uma tupla com o estado(uf) e o elemento
  """
  chave = elemento['uf']
  return (chave, elemento)

In [ ]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) #pular a primeira linha com o skip_header
    | "De texto para lista"  >> beam.Map(TextoParaLista)
    | "Lista para dicionário" >> beam.Map(ListaParaDicionario, colunas_dengue)
    | "Criar campo ano-mês" >> beam.Map(TrataDatas)
    | "Criar chave pelo estado" >> beam.Map(ChaveUf)
    | "Mostrar resultados" >> beam.Map(print) 

)

pipeline.run()

Agora precisamos agrupor por estado usando o GroupByKey. Isto é, ele vai reunir todos os dicionários que tenham a mesma chave.

In [ ]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) #pular a primeira linha com o skip_header
    | "De texto para lista"  >> beam.Map(TextoParaLista)
    | "Lista para dicionário" >> beam.Map(ListaParaDicionario, colunas_dengue)
    | "Criar campo ano-mês" >> beam.Map(TrataDatas)
    | "Criar chave pelo estado" >> beam.Map(ChaveUf)
    | "Agrupor por estado" >> beam.GroupByKey()
    | "Mostrar resultados" >> beam.Map(print) 

)

pipeline.run()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



('RS', [{'id': '209508', 'data_iniSE': '2014-12-28', 'casos': '2.0', 'ibge_code': '431490', 'cidade': 'Porto Alegre', 'uf': 'RS', 'cep': '90000-000', 'latitude': '-30.1146', 'longitude': '-51.1639', 'ano-mes': '2014-12'}, {'id': '209509', 'data_iniSE': '2015-01-04', 'casos': '5.0', 'ibge_code': '431490', 'cidade': 'Porto Alegre', 'uf': 'RS', 'cep': '90000-000', 'latitude': '-30.1146', 'longitude': '-51.1639', 'ano-mes': '2015-01'}, {'id': '209510', 'data_iniSE': '2015-01-11', 'casos': '6.0', 'ibge_code': '431490', 'cidade': 'Porto Alegre', 'uf': 'RS', 'cep': '90000-000', 'latitude': '-30.1146', 'longitude': '-51.1639', 'ano-mes': '2015-01'}, {'id': '209511', 'data_iniSE': '2015-01-18', 'casos': '9.0', 'ibge_code': '431490', 'cidade': 'Porto Alegre', 'uf': 'RS', 'cep': '90000-000', 'latitude': '-30.1146', 'longitude': '-51.1639', 'ano-mes': '2015-01'}, {'id': '209512', 'data_iniSE': '2015-01-25', 'casos': '6.0', 'ibge_code': '431490', 'cidade': 'Porto Alegre', 'uf': 'RS', 'cep': '90000-

ossa pcollection de dengue resultou do último tratamento uma tupla. Essa tupla tem uma chave, que é o primeiro elemento da minha tupla, que é o estado. E nós temos o segundo elemento, que é um array.

Esse array vai ter diversos dicionários, cada dicionário com um conjunto de dados que seriam aquela linha inicial, cada linha que foi lida do nosso arquivo. O que precisamos fazer é transformar essa tupla agora acrescentando a chave, que seria o ano e mês, lembrem disso da nossa fase de análise.

 Então nós vamos acrescentar o campo ano e mês ao estado e outra coisa que também nós vamos fazer é remover os campos, nós vamos deixar apenas a quantidade de casos aqui, que é que nós vamos utilizar, e a nossa chave composta pelo estado, ano e mês.
 
 Outra coisa que nós temos que fazer é descompactar os dados. O que seria esse descompactar os dados? Nós temos aqui uma lista, um array com várias quantidades de casos para cada elemento desses, então o que nós vamos retornar é uma tupla. Para cada elemento desse array nós vamos retornar uma tupla contendo a chave, no caso de todo o agrupamento que nós criamos, e a quantidade de casos. 

In [ ]:
def CasosDengue(elemento):
  """
  Recebe uma tupla (RS, [{},{}])
  Retorna uma tupla ('RS-2014-12, 8')
  """
  uf, registros = elemento
  for registro in registros:
    if bool(re.search(r'\d', registro['casos'])):
      yield (f"{uf}-{registro['ano-mes']}", float(registro['casos'])) #Diferente do return o yield vai executar até retornar todos os elementos
    else:
      yield (f"{uf}-{registro['ano-mes']}", 0.0)

In [ ]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) #pular a primeira linha com o skip_header
    | "De texto para lista"  >> beam.Map(TextoParaLista)
    | "Lista para dicionário" >> beam.Map(ListaParaDicionario, colunas_dengue)
    | "Criar campo ano-mês" >> beam.Map(TrataDatas)
    | "Criar chave pelo estado" >> beam.Map(ChaveUf)
    | "Agrupor por estado" >> beam.GroupByKey()
    | "Descompactar casos de dengue" >> beam.FlatMap(CasosDengue)
    | "Mostrar resultados" >> beam.Map(print) 

)

pipeline.run()

Agora vamos retornar para cada chave a soma dos casos

In [ ]:
dengue = (
    pipeline
    | "Leitura do dataset de dengue" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/casos_dengue.txt', skip_header_lines=1) #pular a primeira linha com o skip_header
    | "De texto para lista"  >> beam.Map(TextoParaLista)
    | "Lista para dicionário" >> beam.Map(ListaParaDicionario, colunas_dengue)
    | "Criar campo ano-mês" >> beam.Map(TrataDatas)
    | "Criar chave pelo estado" >> beam.Map(ChaveUf)
    | "Agrupor por estado" >> beam.GroupByKey()
    | "Descompactar casos de dengue" >> beam.FlatMap(CasosDengue)
    | "Soma dos casos pela chave" >> beam.CombinePerKey(sum) #lembrar de converter os dados para ponto flutuante
    | "Mostrar resultados" >> beam.Map(print) 

)

pipeline.run()

# Tratamento para o arquivo Chuvas

Toda a pipeline de dengue recebeu todo esse tratamento a cima. Agora faremos algo similar para a pipeline chuvas

In [ ]:
chuvas = (
    pipeline
    | "Leitura do dataset de chuvas" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/chuvas.csv', skip_header_lines=1)
    | "De texto para lista (chuvas)"  >> beam.Map(TextoParaLista,delimitador = ',') #reutilizando o método anterior só mudando separador
    | "Mostrar resultados" >> beam.Map(print) 
)

pipeline.run()

precisamos igualar o formato dos pipelines chuvas e dengue
Isto é, o pipeline chuvas deverá estar no formato. Antes de somarmos todos os casos de chuvas devemos tratar os dados pois há inconsistências, verificando o dataset de chuvas verá que alguns dias possuem valor -9999 e isso é um erro que devemos corrigir.


In [ ]:
def ChaveUfMes(elemento):
  """
  Receber uma lista de elementos
  Retorna uma tupla ('UF-ANO-MES', casos)
  """
  data, mm, uf = elemento
  ano_mes = '-'.join(data.split('-')[:2])
  chave = f'{uf}-{ano_mes}'
  if float(mm) < 0:
    mm = 0.0
  else:
    mm = float(mm)
  return chave, mm

In [ ]:
#Caso você ainda esteja em duviada de como o processo acima funciona, observe o exemplo
lista = ['2022-08-10', '4.2', 'TO']
dt, mim, uf1 = lista
print(dt)
print(mim)
print(uf1)
anomes = '-'.join(dt.split('-')[:2])
key = f'{uf1}-{anomes}'
print(f'Teremos os seguinte resultado: {key}')

2022-08-10
4.2
TO
Teremos os seguinte resultado: TO-2022-08


In [ ]:
chuvas = (
    pipeline
    | "Leitura do dataset de chuvas" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/chuvas.csv', skip_header_lines=1)
    | "De texto para lista (chuvas)"  >> beam.Map(TextoParaLista,delimitador = ',') #reutilizando o método anterior só mudando separador
    | "Criando chave UF-ANO-MES" >> beam.Map(ChaveUfMes)
    | "Mostrar resultados" >> beam.Map(print) 
)

pipeline.run()

Agora sim, podemos somar todos os casos.

In [ ]:
chuvas = (
    pipeline
    | "Leitura do dataset de chuvas" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/chuvas.csv', skip_header_lines=1)
    | "De texto para lista (chuvas)"  >> beam.Map(TextoParaLista,delimitador = ',') #reutilizando o método anterior só mudando separador
    | "Criando chave UF-ANO-MES" >> beam.Map(ChaveUfMes)
    | "Combinando o total de chuvas pela chave" >> beam.CombinePerKey(sum)
    | "Mostrar resultados" >> beam.Map(print) 
)

pipeline.run()

('PA-2015-09', 252.1999999999999)
('PA-2015-10', 546.599999999999)
('PA-2015-11', 358.1999999999992)
('PA-2015-12', 1122.6000000000038)
('SP-2015-01', 4464.999999999963)
('SP-2015-02', 6594.999999999894)
('SP-2015-03', 6210.599999999847)
('SP-2015-04', 1862.0000000000134)
('SP-2015-05', 2756.599999999995)
('SP-2015-06', 571.1999999999989)
('SP-2015-07', 2145.400000000012)
('SP-2015-08', 568.3999999999987)
('SP-2015-09', 4513.799999999931)
('SP-2015-10', 3177.599999999992)
('SP-2015-11', 7333.1999999998725)
('SP-2015-12', 6328.59999999989)
('MG-2015-01', 3511.3999999999837)
('MG-2015-02', 8882.39999999986)
('MG-2015-03', 8544.199999999848)
('MG-2015-04', 3812.9999999999486)
('MG-2015-05', 2958.399999999979)
('MG-2015-06', 841.2000000000071)
('MG-2015-07', 486.3999999999967)
('MG-2015-08', 479.7999999999971)
('MG-2015-09', 2978.19999999999)
('MG-2015-10', 3052.3999999999837)
('MG-2015-11', 9413.399999999967)
('MG-2015-12', 8446.59999999986)
('SC-2015-01', 4375.599999999942)
('SC-2015-02'

Agora temos uma quantidade de dados muito menor que anteriormente, mas ainda tempos um problema que é a quantidade de casas decimais vinda do float. Iremos corrigir esse problema criando um outro método:

In [ ]:
def arredonda(elemento):
  """
  Recebe um elemento 
  retorna o float com uma ou duas casas decimais.
  """
  chave, mm = elemento
  return (chave, round(mm,1))

In [ ]:
chuvas = (
    pipeline
    | "Leitura do dataset de chuvas" >>
      ReadFromText('/content/drive/MyDrive/Colab Notebooks/Dados/chuvas.csv', skip_header_lines=1)
    | "De texto para lista (chuvas)"  >> beam.Map(TextoParaLista,delimitador = ',') #reutilizando o método anterior só mudando separador
    | "Criando chave UF-ANO-MES" >> beam.Map(ChaveUfMes)
    | "Combinando o total de chuvas pela chave" >> beam.CombinePerKey(sum)
    | "Arredondar resultados de chuvas" >> beam.Map(arredonda)
    | "Mostrar resultados" >> beam.Map(print) 
)

pipeline.run()

Para os próximos passos, vá para o arquivo 2.Chuvas e Dengue - Analise